# Machine Learning to predict Down Jones Industrial Average

This simple Machine Learning example shows how to predict [^DJI value](https://finance.yahoo.com/quote/%5EDJI?p=^DJI&.tsrc=fin-srch) based on the past calculated averages.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Setup" data-toc-modified-id="Setup-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Setup</a></span></li><li><span><a href="#Read-data-into-a-SFrame" data-toc-modified-id="Read-data-into-a-SFrame-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Read data into a SFrame</a></span><ul class="toc-item"><li><span><a href="#TODO:-Value-should-be-original-value" data-toc-modified-id="TODO:-Value-should-be-original-value-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>TODO: Value should be original value</a></span></li></ul></li><li><span><a href="#Select-the-data-to-train-and-test" data-toc-modified-id="Select-the-data-to-train-and-test-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Select the data to train and test</a></span><ul class="toc-item"><li><span><a href="#TODO:-Let's-NOT-take-last-few-days" data-toc-modified-id="TODO:-Let's-NOT-take-last-few-days-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>TODO: Let's NOT take last few days</a></span></li></ul></li><li><span><a href="#Create-the-model" data-toc-modified-id="Create-the-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Create the model</a></span><ul class="toc-item"><li><span><a href="#Print-example-predictions" data-toc-modified-id="Print-example-predictions-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Print example predictions</a></span></li></ul></li><li><span><a href="#&quot;Be-Less-Wrong&quot;" data-toc-modified-id="&quot;Be-Less-Wrong&quot;-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>"Be Less Wrong"</a></span><ul class="toc-item"><li><span><a href="#Previous-results:" data-toc-modified-id="Previous-results:-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Previous results:</a></span><ul class="toc-item"><li><span><a href="#^DJI-averages-only" data-toc-modified-id="^DJI-averages-only-5.1.1"><span class="toc-item-num">5.1.1&nbsp;&nbsp;</span>^DJI averages only</a></span></li><li><span><a href="#added-ISM-Manufacturing-Employment" data-toc-modified-id="added-ISM-Manufacturing-Employment-5.1.2"><span class="toc-item-num">5.1.2&nbsp;&nbsp;</span>added ISM Manufacturing Employment</a></span></li><li><span><a href="#added-US-Housing-Starts-m/m" data-toc-modified-id="added-US-Housing-Starts-m/m-5.1.3"><span class="toc-item-num">5.1.3&nbsp;&nbsp;</span>added US Housing Starts m/m</a></span></li><li><span><a href="#added-Manufacturing-PMI" data-toc-modified-id="added-Manufacturing-PMI-5.1.4"><span class="toc-item-num">5.1.4&nbsp;&nbsp;</span>added Manufacturing PMI</a></span></li></ul></li><li><span><a href="#TODO:-find-the-best-model" data-toc-modified-id="TODO:-find-the-best-model-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>TODO: find the best model</a></span></li></ul></li><li><span><a href="#Save-the-model" data-toc-modified-id="Save-the-model-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Save the model</a></span></li></ul></div>

## Setup

In [1]:
column_to_predict = "DJIA_Original"

# Location of the spreadsheet (Comma Delimited Value) with all info that I prepared in a separate notebook.
data_path="./DATA/processed/uber.csv"

# Install TuriCreate. Last updated November 4, 2020

# !pip install --upgrade pip
# !pip install Turicreate

import turicreate as tc

## Read data into a SFrame

In [2]:
# Load the data
data =  tc.SFrame(data_path)
data[364:370] # show data sample

Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/uber.csv

Parsing completed. Parsed 100 lines in 0.028903 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,str,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/uki/REPOS/MarketIndicators.jl/DATA/processed/uber.csv

Parsing completed. Parsed 1744 lines in 0.011634 secs.

Columns:
	Day	int
	Date	str
	DJIA_Value	float
	DJIA_Original	float
	DJIA_Avg005	float
	DJIA_Avg030	float
	DJIA_Avg090	float
	DJIA_Avg180	float
	DJIA_Avg365	float
	ISM_MFC_EMP_Value	float
	ISM_MFC_EMP_Original	float
	ISM_MFC_EMP_Avg090	float
	ISM_MFC_EMP_Avg180	float
	ISM_MFC_EMP_Avg365	float
	HOUSE_SRT_MM_Value	float
	HOUSE_SRT_MM_Original	float
	HOUSE_SRT_MM_Avg090	float
	HOUSE_SRT_MM_Avg180	float
	HOUSE_SRT_MM_Avg365	float
	MFC_MPI_Value	float
	MFC_MPI_Original	float
	MFC_MPI_Avg090	float
	MFC_MPI_Avg180	float
	MFC_MPI_Avg365	float

Rows: 6

Data:
+--------+------------+------------+---------------+-------------+-------------+
|  Day   |    Date    | DJIA_Value | DJIA_Original | DJIA_Avg005 | DJIA_Avg030 |
+--------+------------+------------+---------------+-------------+-------------+
| 736339 | 2017-01-10 |    38.0    |  19957.119141 |     38.6    |    38.07    |
| 736340 | 2017-01-11 |    38.0    |  19973.419922 |     38.4    |     38.1    |
| 736341 | 2017-01-12 |    38.0    |  19929.289063 |     38.2    |     38.1    |
| 736342 | 2017-01-13 |    38.0    |  19952.029297 |     38.0    |     38.1    |
| 736343 | 2017-01-14 |    38.0    |  19952.029297 |     38.0    |     38.1    |
| 736344 | 2017-01-15 |    38.0    |  19952.029297 |     38.0    |     38.1    |
+--------+------------+------------+---------------+-------------+-------------+
+-------------+-------------+-------------+-------------------+----------------------+
| DJIA_Avg090 | DJIA_Avg180 | DJIA_Avg365 | ISM_MFC_EMP_Value | ISM_MFC_EMP_Original |
+-------------+-------------+-------------+-------------------+----------------------+
|    30.84    |     27.9    |    21.57    |        71.0       |         53.1         |
|    31.02    |    27.97    |    21.65    |        71.0       |         53.1         |
|    31.19    |    28.03    |    21.73    |        71.0       |         53.1         |
|    31.36    |     28.1    |    21.82    |        71.0       |         53.1         |
|    31.52    |    28.17    |     21.9    |        71.0       |         53.1         |
|     31.7    |    28.23    |    21.99    |        71.0       |         53.1         |
+-------------+-------------+-------------+-------------------+----------------------+
+--------------------+--------------------+--------------------+--------------------+
| ISM_MFC_EMP_Avg090 | ISM_MFC_EMP_Avg180 | ISM_MFC_EMP_Avg365 | HOUSE_SRT_MM_Value |
+--------------------+--------------------+--------------------+--------------------+
|       62.64        |       51.82        |       42.85        |       -75.0        |
|       62.93        |       51.94        |       42.96        |       -75.0        |
|       63.22        |       52.06        |       43.06        |       -75.0        |
|       63.51        |       52.17        |       43.17        |       -75.0        |
|        63.8        |       52.29        |       43.28        |       -75.0        |
|       64.09        |       52.41        |       43.38        |       -75.0        |
+--------------------+--------------------+--------------------+--------------------+
+-----------------------+---------------------+---------------------+---------------------+
| HOUSE_SRT_MM_Original | HOUSE_SRT_MM_Avg090 | HOUSE_SRT_MM_Avg180 | HOUSE_SRT_MM_Avg365 |
+-----------------------+---------------------+---------------------+---------------------+
|         -18.7         |         8.2         |        11.12        |         9.8         |
|         -18.7         |         7.54        |        10.66        |         9.6         |
|         -18.7         |         6.89        |        10.19        |         9.4         |
|         -18.7         |         6.23        |         9.72        |         9.19        |
|         -18.7         |         5.58        |         9.26        |         8.99        |
|         -18.7         |         4.92        |         8.66        |         8.79        |
+-----------------------+---------------------+---------------------+-------

### TODO: Value should be original value

Please note the the "High" is normalized to Int8, 
but for the prediciton purposes it should be an original "real" value.

## Select the data to train and test

In [3]:
row_count = len(data)
# Do not take initial year data as averages are not complete
data = data[365:row_count] 
# Make a train-test split
train_data, test_data = data.random_split(0.8)

### TODO: Let's NOT take last few days

I need to save the last few days to see if I can really predict upcoming values.

## Create the model

- https://apple.github.io/turicreate/docs/api/generated/turicreate.regression.create.html
- Automatically picks the right model based on your data.
- target: is the number to be predicted.
- features: are the the values that we ues to try to find pattern leading to prediciton.

In [4]:
model = tc.regression.create(
    train_data, 
    target = column_to_predict,
    features = [
         'DJIA_Avg005'
        ,'DJIA_Avg030'
        ,'DJIA_Avg090'
        ,'DJIA_Avg180'
        ,'DJIA_Avg365'
        
        ,'ISM_MFC_EMP_Avg090'
        ,'ISM_MFC_EMP_Avg180'
        ,'ISM_MFC_EMP_Avg365'
        
        ,'HOUSE_SRT_MM_Value'
        ,'HOUSE_SRT_MM_Avg090'
        ,'HOUSE_SRT_MM_Avg180'
        ,'HOUSE_SRT_MM_Avg365'
        
        ,'MFC_MPI_Value'
        ,'MFC_MPI_Avg090'
        ,'MFC_MPI_Avg180'
        ,'MFC_MPI_Avg365'
    ],
    validation_set='auto', 
    verbose=True
)

Automatically generating validation set from 5% of the data.

Linear regression:

--------------------------------------------------------

Number of examples          : 1050

Number of features          : 16

...

| 3         | 0.009222     | 11259.470703       | 10457.419922         | 8676.626953                     | 8332.853516                       |

| 4         | 0.012330     | 8249.074219        | 7447.023438          | 6090.535156                     | 5845.973145                       |

| 5         | 0.015705     | 6154.664062        | 5352.613281          | 4278.541016                     | 4072.104980                       |

| 10        | 0.033702     | 1988.382812        | 1287.683594          | 766.906921                      | 719.447083                        |

+-----------+--------------+--------------------+----------------------+---------------------------------+-----------------------------------+

# Predict values on data that was NOT used in training

In [5]:
#test_data.explore()
test_data

Day,Date,DJIA_Value,DJIA_Original,DJIA_Avg005,DJIA_Avg030,DJIA_Avg090,DJIA_Avg180,DJIA_Avg365
736350,2017-01-21,37.0,19843.939453,37.2,37.9,32.69,28.61,22.53
736358,2017-01-29,40.0,20115.970703,39.8,38.1,34.1,29.22,23.27
736361,2017-02-01,38.0,19967.730469,39.0,38.23,34.67,29.45,23.53
736371,2017-02-11,41.0,20298.210938,40.4,38.7,36.27,30.2,24.47
736373,2017-02-13,43.0,20441.480469,41.4,38.97,36.56,30.38,24.68
736383,2017-02-23,46.0,20840.699219,45.0,41.3,38.19,31.4,25.71
736391,2017-03-03,48.0,21039.960938,47.6,43.37,39.57,32.35,26.51
736398,2017-03-10,47.0,20940.289063,47.0,45.27,40.57,33.22,27.17
736401,2017-03-13,47.0,20926.060547,47.0,45.87,40.89,33.62,27.44
736404,2017-03-16,48.0,21000.109375,47.0,46.33,41.19,34.02,27.71


In [6]:
## Save predictions to an SArray
predictions = model.predict(test_data)
#predictions

### Print example predictions

In [7]:
start = 0
end = len(predictions)
step = 30

print(column_to_predict)

for id in range(start, end, step):
    a = round( predictions[id], 2)
    b = test_data[id][column_to_predict]
    print( "predicted ", round(a, 0), "\t, but actual value was \t", round(b, 0) , "\t difference is \t", round(b-a, 2) ) # dict

DJIA_Original
predicted  19916.0 	, but actual value was 	 19844.0 	 difference is 	 -72.2
predicted  21411.0 	, but actual value was 	 21367.0 	 difference is 	 -43.26
predicted  21958.0 	, but actual value was 	 21847.0 	 difference is 	 -110.94
predicted  25483.0 	, but actual value was 	 25800.0 	 difference is 	 317.54
predicted  24477.0 	, but actual value was 	 24520.0 	 difference is 	 43.21
predicted  25481.0 	, but actual value was 	 25691.0 	 difference is 	 210.16
predicted  26526.0 	, but actual value was 	 26602.0 	 difference is 	 76.79
predicted  26649.0 	, but actual value was 	 26591.0 	 difference is 	 -58.22
predicted  21201.0 	, but actual value was 	 20443.0 	 difference is 	 -758.31
predicted  26885.0 	, but actual value was 	 27025.0 	 difference is 	 139.99


## "Be Less Wrong"

Evaluate how good is the model

It appears that the predition results vary from run to run so it is worth to run it until you find the model with minimum error, 

or **as Elon Musk says "Be less wrong"**.

### Previous results:

#### ^DJI averages only 

- {'max_error': 1749.5078773959249, 'rmse': 124.58897796835019}
- {'max_error': 1621.9227669335778, 'rmse': 106.39104997423203}
- {'max_error': 1297.117071650111, 'rmse': 101.14871945325757} - BEST RMSE
- {'max_error': 1122.2711616305896, 'rmse': 183.129076342891}


#### added ISM Manufacturing Employment
- {'max_error': 1708.487399827758, 'rmse': 235.1824022060072}
- {'max_error': 1093.9698394310544, 'rmse': 188.48468898003293} - BEST Max Error

#### added US Housing Starts m/m
- {'max_error': 1715.473257380545, 'rmse': 255.97228904279297}
- {'max_error': 1255.8992158671826, 'rmse': 217.41264660030447}
- {'max_error': 1102.1036788719757, 'rmse': 226.13843475657265}
- {'max_error': 1397.611965987675, 'rmse': 236.1374235983197}

#### added Manufacturing PMI
- {'max_error': 1522.3008456494535, 'rmse': 236.8009135100335}
- {'max_error': 1629.302702719142, 'rmse': 229.67414881442798}

### TODO: find the best model

Create a "for" loop to find the best model

In [8]:
#TODO: write this in a loop to select the best model
# Evaluate the model and save the results into a dictionary
results = model.evaluate( test_data ) #test_data[0:2531]
results

{'max_error': 1073.0490379783332, 'rmse': 220.87504305899103}

## Save the model

Save the model for future use in MacOS, iOS, etc. applications

In [9]:
# Export to Core ML
model.export_coreml('./DATA/models/^DJI.mlmodel')